### Graph transformation history

In [ ]:
"""Experiments with non sequential transformations"""
import functools
import ast
import nographs as nog
from graphviz import Digraph

In [ ]:
dict_2= {0:{1,}, 1:{2,3}, 2:{4,}, 3:{}, 4:{} }

In [ ]:
def to_id(vert):
    return f"{(str( vert ))}"

node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#landscape="True",layout="neato"

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @functools.wraps( wrapped )
    def wrapper_bg( vert, _trav ):
        dot.node( to_id(vert), str(vert) )
        for result in wrapped(vert, _trav):
            dot.edge( to_id(vert), to_id( result ), )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

In [ ]:
class GraphMorph:
    # pylint: disable=no-member   #(__wrapped__)
    """identity morpher for start"""
    def __init__(self, wrapped):
        functools.update_wrapper(self, wrapped )
    def __call__(self, vert, _trav):
        yield from self.__wrapped__( vert, _trav )

class GraphMorphDecorateNodes(GraphMorph):
    # pylint: disable=no-member   #(__wrapped__)
    """morpher for node names base"""
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( vert,):
        return vert
    @staticmethod
    def undress( vert,):
        return vert
    def __call__(self, vert, _trav):
        yield from map( self.dress, self.__wrapped__( self.undress(vert), _trav ) )

class DecoName(GraphMorphDecorateNodes):
    """morpher for node names example"""
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( vert,):
        return f"-{vert}-"
    @staticmethod
    def undress( vert,):
        return ast.literal_eval(vert[1:-1])

In [ ]:
class FreeForget(GraphMorph):  # All possible paths
    # pylint: disable=no-member   #(__wrapped__)
    """new graph - all paths of old one"""
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    def __call__(self, vert, _trav):
        yield vert # identity morphism
        for ch1 in self.__wrapped__( vert, _trav ):
            yield from self.__call__(ch1, _trav) # recursion

In [ ]:
class IDFunctor(GraphMorphDecorateNodes):
    # pylint: disable=no-member   #(__wrapped__)
    """new graph - two copies of old one with arrows from one to another"""
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( vert,):
        return f"-{vert}-"
    @staticmethod
    def undress( vert,):
        return ast.literal_eval(vert[1:-1])
    def __call__(self, vert, _trav):
        if type(vert) is int:
            yield from                  self.__wrapped__(              vert,  _trav )
            yield self.dress(vert)
        else:
            yield from map( self.dress, self.__wrapped__( self.undress(vert), _trav ) )
            #yield from                  self.__wrapped__( self.undress(vert), _trav )

In [ ]:
class FreeForgetFunctor(GraphMorphDecorateNodes):
    # pylint: disable=no-member   #(__wrapped__)
    """new graph - old one projected to `FreeForget`"""
    def __init__(self, wrapped ):
        super().__init__(wrapped)
    @staticmethod
    def dress( vert,):
        return f"-{vert}-"
    @staticmethod
    def undress( vert,):
        return ast.literal_eval(vert[1:-1])
    def ff(self, vert, _trav):
        yield vert # identity morphism
        for ch1 in self.__wrapped__( vert, _trav ):
            yield from self.ff(ch1, _trav) # recursion
    def __call__(self, vert, _trav):
        if type(vert) is int:
            yield from                  self.__wrapped__(              vert,  _trav )
            yield self.dress(vert)
        else:
            yield from map( self.dress, self.ff( self.undress(vert), _trav ) )

In [ ]:
class graphviz_bipart(  ):
    def __init__(self, predicate ):
        self.dot = {}
        self.dot[(False,False)] = Digraph( name="cluster_0" )
        self.dot[(True, True) ] = Digraph( name="cluster_1" )
        self.dot[(False,True) ] = Digraph( name='master')
        self.p = predicate
    def finalize(self):
        self.dot[(False,True) ].subgraph(self.dot[(False,False)])
        self.dot[(False,True) ].subgraph(self.dot[(True, True) ])
        return  self
    def get_dot(self):
        return  self.finalize().dot[(False,True) ]
    def __call__( self, to_wrap ):
        @functools.wraps(to_wrap)
        def wrapper( vert,_trav ):
            self.dot[self.p(vert),self.p(vert)].node( to_id(vert), str(vert) )
            for result in to_wrap(vert, _trav):
                colour = 'black' if self.p(vert) == self.p(result) else 'red'
                self.dot[(self.p(result),self.p(vert))
                            ].edge( to_id(vert), to_id(result), color=colour)
                yield result
        wrapper.get_dot = self.get_dot
        return wrapper

In [ ]:
def cat_dog():
    @graphviz_bipart( lambda vert: type(vert) is int )
    @FreeForgetFunctor
    def cat(vert, _):
        return dict_2[vert]
    return cat

cat_3 = cat_dog()
trav_forward = nog.TraversalBreadthFirst( cat_3 )
trav_forward.start_from( 0, )
print(str( list(trav_forward) ))
the_dot = cat_3.get_dot()
the_dot              # pylint: disable=pointless-statement

In [ ]:
class DecoNamePlus(DecoName):
    def __init__(self,*argc,**argv):
        super().__init__(*argc,**argv)
    @staticmethod
    def dress( vert,):
        return f"+{vert}+"

In [ ]:
def cat_dog_cat():
    @graphviz_bipart( lambda v: type(v) is int )
    # @DecoNamePlus
    @FreeForgetFunctor
    def cat(vert, _):
        return dict_2[vert]
    return cat

cat_4 = cat_dog_cat()
trav_forward = nog.TraversalBreadthFirst( cat_4 )
trav_forward.start_from( '.0.', )
print(str( list(trav_forward) ))
the_dot = cat_4.get_dot()
the_dot              # pylint: disable=pointless-statement